In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, BatchNormalization
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import regularizers
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Parámetros de la red
EPOCHS = 50
IMAGE_SIZE = (128, 128)
INPUT_SHAPE = (128, 128, 3)
SEED = 123
BATCH_SIZE = 32
BUFFER_SIZE = 250
LEARNING_RATE = 0.001

# Directorio de las imágenes
images_dir = '../arcgis-survey-images'

# Cargar datasets de entrenamiento y validación
train_ds = image_dataset_from_directory(
    images_dir,
    labels="inferred",
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    shuffle=True
)

validation_ds = image_dataset_from_directory(
    images_dir,
    labels="inferred",
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    validation_split=0.2,
    subset="validation",
    seed=SEED
)

class_names = train_ds.class_names
num_classes = len(class_names)

# Función para contar ejemplos en un dataset
def count_examples(dataset):
    return dataset.reduce(0, lambda x, _: x + 1).numpy()

# Manejo del Desbalanceo de Clases mediante Sobremuestreo
def oversample_dataset(dataset, majority_size):
    return dataset.repeat().take(majority_size)

# Filtrar cada clase
def filter_class(dataset, class_label):
    return dataset.filter(lambda x, y: tf.reduce_any(tf.equal(y, class_label)))

# Contar ejemplos por clase
def get_class_sizes(dataset, num_classes):
    sizes = []
    for i in range(num_classes):
        class_ds = filter_class(dataset, i)
        size = count_examples(class_ds)
        sizes.append(size)
    return sizes

# Obtener el tamaño de la clase mayoritaria
train_class_sizes = get_class_sizes(train_ds, num_classes)
majority_class_size = max(train_class_sizes)

# Sobremuestrear cada clase
oversampled_datasets = []
for i in range(num_classes):
    class_ds = filter_class(train_ds, i)
    oversampled_ds = oversample_dataset(class_ds, majority_class_size)
    oversampled_datasets.append(oversampled_ds)

# Concatenar los datasets sobremuestreados
oversampled_train_ds = oversampled_datasets[0]
for ds in oversampled_datasets[1:]:
    oversampled_train_ds = oversampled_train_ds.concatenate(ds)

# Aplicar optimización de cache y prefetch
oversampled_train_ds = oversampled_train_ds.cache().shuffle(BUFFER_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_ds = validation_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# Verificar la nueva distribución de clases
print("Nueva distribución de clases después del sobremuestreo:")
for i, class_size in enumerate(get_class_sizes(oversampled_train_ds, num_classes)):
    print(f"{class_names[i]}: {class_size}")

# Aumento de Datos (Data Augmentation)
data_augmentation = tf.keras.Sequential([
    RandomFlip("horizontal_and_vertical"),
    RandomRotation(0.2),
    RandomZoom(0.2),
])

# Preprocesamiento de Imágenes
def preprocess_image(image, label):
    # Normalizar imágenes
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Aplicar aumento de datos y preprocesamiento solo al conjunto de entrenamiento
def prepare_train_ds(ds):
    ds = ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.cache().shuffle(BUFFER_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

def prepare_val_ds(ds):
    ds = ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

oversampled_train_ds = prepare_train_ds(oversampled_train_ds)
validation_ds = prepare_val_ds(validation_ds)

# Obtener las etiquetas de entrenamiento para calcular los pesos de clase
y_train = np.concatenate([y for x, y in train_ds], axis=0)

# Calcular los pesos de clase
class_weights_values = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_dict = dict(enumerate(class_weights_values))
print("Pesos de clase:", class_weights_dict)

# Cargar el modelo base (EfficientNetB0) con pesos preentrenados
base_model = EfficientNetB0(input_shape=INPUT_SHAPE,
                            include_top=False,
                            weights='imagenet')

# Congelar las primeras capas del modelo base
for layer in base_model.layers[:100]:  # Ajusta este número según sea necesario
    layer.trainable = False

# Definir el modelo
model = tf.keras.models.Sequential([
    base_model,
    BatchNormalization(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')  # Usar softmax para SparseCategoricalCrossentropy
])

# Compilar el modelo
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=LEARNING_RATE,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Callbacks para mejorar el entrenamiento
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5)
]

# Entrenamiento del modelo
history = model.fit(
    oversampled_train_ds,
    validation_data=validation_ds,
    epochs=EPOCHS,
    class_weight=class_weights_dict,
    callbacks=callbacks
)

# Gráfica de la pérdida y precisión
def plot_history(history):
    metrics = history.history
    plt.figure(figsize=(16, 6))
    
    # Pérdida
    plt.subplot(1, 2, 1)
    plt.plot(history.epoch, metrics['loss'], label='Pérdida de entrenamiento')
    plt.plot(history.epoch, metrics['val_loss'], label='Pérdida de validación')
    plt.legend()
    plt.ylim([0, max(metrics['loss']) * 1.1])
    plt.ylabel('Pérdida')
    plt.xlabel('Época')
    plt.title('Pérdida durante el Entrenamiento')
    
    # Precisión
    plt.subplot(1, 2, 2)
    plt.plot(history.epoch, metrics['accuracy'], label='Precisión de entrenamiento')
    plt.plot(history.epoch, metrics['val_accuracy'], label='Precisión de validación')
    plt.legend()
    plt.ylim([0, 1])
    plt.ylabel('Precisión')
    plt.xlabel('Época')
    plt.title('Precisión durante el Entrenamiento')
    
    plt.show()

plot_history(history)

# Evaluación en el conjunto de test
test_ds = validation_ds.shard(num_shards=2, index=1)
test_results = model.evaluate(test_ds, return_dict=True)
print("\nResultados de evaluación en test set:")
for metric, value in test_results.items():
    print(f"{metric}: {value:.4f}")

# Predicciones y matriz de confusión
y_pred = model.predict(test_ds)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.concatenate([y for x, y in test_ds], axis=0)

conf_matrix = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, xticklabels=class_names, yticklabels=class_names, annot=True, fmt='g', cmap='Blues')
plt.xlabel('Predicción')
plt.ylabel('Etiqueta')
plt.title('Matriz de Confusión')
plt.show()

# Reporte de clasificación
print("\nReporte de clasificación:")
print(classification_report(y_true, y_pred_classes, target_names=class_names))

# Fine-Tuning del modelo base
# Descongelar todas las capas para fine-tuning
for layer in base_model.layers:
    layer.trainable = True

# Recompilar el modelo con una tasa de aprendizaje más baja
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Entrenamiento adicional para fine-tuning
fine_tune_epochs = 20
total_epochs = EPOCHS + fine_tune_epochs

history_finetune = model.fit(
    oversampled_train_ds,
    validation_data=validation_ds,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    class_weight=class_weights_dict,
    callbacks=callbacks
)

# Gráfica de la pérdida y precisión después del fine-tuning
plot_history(history_finetune)

# Evaluación final en el conjunto de test
final_test_results = model.evaluate(test_ds, return_dict=True)
print("\nResultados de evaluación final en test set después del fine-tuning:")
for metric, value in final_test_results.items():
    print(f"{metric}: {value:.4f}")

# Guardar el modelo final
model.save('final_model.keras')
print("\nModelo final guardado como 'final_model.keras'")


Found 3289 files belonging to 5 classes.
Using 2632 files for training.
Found 3289 files belonging to 5 classes.
Using 657 files for validation.
Nueva distribución de clases después del sobremuestreo:
Chinche salivosa: 414
Clororis: 413
Hoja sana: 415
Roya naranja: 415
Roya purpura: 383
Pesos de clase: {0: 0.7963691376701967, 1: 1.6097859327217126, 2: 0.7927710843373494, 3: 0.6588235294117647, 4: 2.9082872928176795}
Epoch 1/50
415/415 ━━━━━━━━━━━━━━━━━━━━ 230s 488ms/step - accuracy: 0.2842 - loss: 4.0439 - val_accuracy: 0.2496 - val_loss: 2.5527 - learning_rate: 0.0010
Epoch 2/50
415/415 ━━━━━━━━━━━━━━━━━━━━ 199s 478ms/step - accuracy: 0.4415 - loss: 2.0104 - val_accuracy: 0.2481 - val_loss: 2.1427 - learning_rate: 0.0010
Epoch 3/50
415/415 ━━━━━━━━━━━━━━━━━━━━ 197s 474ms/step - accuracy: 0.5017 - loss: 1.5412 - val_accuracy: 0.1963 - val_loss: 1.6614 - learning_rate: 0.0010
Epoch 4/50
415/415 ━━━━━━━━━━━━━━━━━━━━ 195s 470ms/step - accuracy: 0.5284 - loss: 1.4210 - val_accuracy: 0.3212